# 1) Conversion of CSV Prediction Results to Geotiff Format

    This code processes CSV files containing predicted classifications with spatial coordinates, converts them into geospatial dataframes using GeoPandas and Shapely, and then exports these as geotiff raster files using the geocube library, allowing spatial analysis and visualization of prediction results.

In [ ]:
import os
import time
import pandas as pd
import geopandas as gpd
import shapely
from geocube.api.core import make_geocube

csv_folder = '/path/to/prediction_result_csv/'
result_tif = '/path/to/save/prediction_result_tif/'

start = time.process_time()

csv_files = [file for file in os.listdir(csv_folder) if file.startswith("predicted") and file.endswith(".csv")]

for csv_file in csv_files:
    csv_path = os.path.join(csv_folder, csv_file)
    print("Processing:", csv_path)
    s = time.process_time()
    try:
        df = pd.read_csv(csv_path)
        
        grid_prediction_gdf = gpd.GeoDataFrame(data=df, 
                            geometry=df['geometry'].apply(shapely.wkt.loads), crs="EPSG:32631")
        
        csv_file_short = csv_file[10:23] 
        geotif_file = os.path.join(result_tif,f"Vector_classification_{csv_file_short}_2022-12.tif")
        
        res = 20
        
        out_grd = make_geocube(vector_data=grid_prediction_gdf, 
                               measurements=["pred_class"], 
                               resolution=(-res, res))

        out_grd["pred_class"].rio.to_raster(geotif_file)
        
        print("Classification result in geotiff write as:", geotif_file)
        
        
    except Exception as e:
        print("Error processing:", csv_path)
        print("Error:", e)
    print("Processing time in [min]",( time.process_time() - s)/60)
    
print("Processing for all CSV files completed")
print("Processing time in [min]", (time.process_time() - start) / 60)

# 2) Merging TIFF Files

    This code snippet utilizes GDAL, a geospatial data abstraction library, to merge multiple TIFF files located in a specified directory into a single TIFF file, intended for applications in geographic information systems (GIS). It collects all TIFF files within a given folder, constructs a list of their paths, and then executes a GDAL command to merge these files, outputting the result to a specified location.

In [ ]:
from osgeo import gdal
import glob
import subprocess
import os

folder = '/path/to/prediction_result_tif/'

tif_files = [file for file in os.listdir(folder) if file.endswith(".tif")]

demList = [os.path.join(csv_folder, file) for file in tif_files]
print(demList)

cmd = "/path/to/gdal_merge.py -ps 20 -20 -o /path/to/save/merged_classification.tif"
subprocess.call(cmd.split()+demList)

print("Merging was successful")